# Symbology mapping using dedicated service - F#

### Overview
Reference data encompasses a wide range of specification data about:
* financial instrument such as asset class, symbol, maturity, etc.
* counterparties such as issuer information.
* pricing such as Open, high low and close prices.

The challenge with reference data is that it tends to be sourced from multiple sources: internal, counterparties and providers.

Reference data dedicated service is a result of various data sources and fields sourcing, validation, cross-checking and normalization.<br>

This sample requests **on-demand** instrument's identifiers and enables fields mapping by calling a dedicated static data service, making available clean data.

### Inputs/outputs
Symbology mapping sample is designed to help you searching and requesting instrument's specification data by using a wide range of commonly used fields.
* It takes instrument specification parameter(s) as per input filter and returns the *entries* matching the request.
* The returned *entries* contain `instrument identifier` data

The samples below expose several ways to query the reference data service in order to retrieve instrument identifier and fields. 
Please refer to any other scenario in order to discover how instrument identifier is used.

### Services used
This sample uses *gRPC requests* in order to retrieve reference data set of fields from the hosted service. The queried endpoint in this script are:
* *StaticDataService*: to directly retrieve reference data objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.StaticData.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*

***

# Run reference data queries

### Step 1: Install packages

In [1]:
#i "nuget: file://~/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Restore sources file://~/.nuget/packages/ https://api.nuget.org/v3/index.json Installed Packages Systemathics.Apis, 0.13.2-pre

In [2]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.StaticData.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [3]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Call reference data service

In [4]:
// instantiate the service
let service = new StaticDataService.StaticDataServiceClient(channel)

### Step 4: Create and process request

#### 4.1 Introduction
The properties one can use as per filter field(s) in order to query the API for equities or futures are the following:
* *Name*, *MIC*, *Ticker*, *Contract code*, *ISIN*, *Cusip*, *market data provider IDs*, etc.

You can combine multiple filters in order to target more accurately your request. Combining filters act as a *AND* operator.

The service returns *entries* matching the request filters.
A reply contains two *arrays* dedicated for *futures* and *equities*. Keep in mind that these arrays might be empty depending on the query filters and the results.
Other shared fields such as <i>`Name`</i> or <i>`Identifier`</i> are there to make it easier to identify the instrument needed (among possible other results).  
Depending on the asset type, there might be additional fields for the aforementioned purpose.

The following code snippets will expose the different ways to query for reference data, with different asset types and various parameters.

#### 4.2 Retrieve reference data - Equities

To request for equities, the user must specify the AssetType as <i>`AssetType.Equity`</i> in the static data request.

##### 4.2.1 Equity - by Name

In [5]:
// Generate request
let request = new StaticDataRequest(AssetType = AssetType.Equity, Name = "Apple")

let reply = service.StaticData(request, headers)
display(reply.Equities)

index Identifier Type Primary Operating Name Currency Country TickSizeRule Index Open Close TimeZone LotSize PointValue Price Volume Time Sources Mapping Bloomberg .. 0 Exchange Ticker Provider XNGS AAPL <null> Equity XNGS XNAS Apple Inc USD US [ { { "tick": 0.0001 }: Price: 0, Tick: 0.0001 }, { { "price": 1, "tick": 0.01 }: Price: 1, Tick: 0.01 } ] [ Dow Jones, Nasdaq 100, Nasdaq Composite, Russell 1000, Russell 3000, Snp 500 ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1 <null> <null> Year Month Day 2022 10 24 41 [ { [Yahoo, AAPL]: Key: Yahoo, Value: AAPL }, { [CIK, 0000320193]: Key: CIK, Value: 0000320193 }, { [FIGI, BBG000B9Y5X2]: Key: FIGI, Value: BBG000B9Y5X2 }, { [FIGIC, BBG000B9XRY4]: Key: FIGIC, Value: BBG000B9XRY4 }, { [Valoren, 908440]: Key: Valoren, Value: 908440 }, { [MorningStar, 0P000000GY]: Key: MorningStar, Value: 0P000000GY }, { [Esignal, AAPL]: Key: Esignal, Value: AAPL }, { [Idc|564, 564|AAPL]: Key: Idc|564, Value: 564|AAPL }, { [Figi, BBG000B9XRY4]: Key: Figi, Value: BBG000B9XRY4 }, { [Figic, BBG000B9XRY4]: Key: Figic, Value: BBG000B9XRY4 }, { [Bloomberg, AAPL US Equity]: Key: Bloomberg, Value: AAPL US Equity }, { [Reuters, AAPL.O]: Key: Reuters, Value: AAPL.O } ] AAPL US Equity 1 Exchange Ticker Provider BATS AAPL <null> Equity XNGS XCBO Apple Inc USD US [ { { "tick": 0.0001 }: Price: 0, Tick: 0.0001 }, { { "price": 1, "tick": 0.01 }: Price: 1, Tick: 0.01 } ] [ Dow Jones, Nasdaq 100, Nasdaq Composite, Russell 1000, Russell 3000, Snp 500 ] Seconds Nanos 34200 0 Seconds Nanos 57600 0 ET 1 1 144.96 3159611 Year Month Day 2022 7 11 1 [ { [Idc|729, 729|AAPL]: Key: Idc|729, Value: 729|AAPL }, { [Figi, BBG000B9Y6P9]: Key: Figi, Value: BBG000B9Y6P9 }, { [Figic, BBG000B9XRY4]: Key: Figic, Value: BBG000B9XRY4 }, { [Bloomberg, AAPL UF Equity]: Key: Bloomberg, Value: AAPL UF Equity } ] AAPL UF Equity 2 Exchange Ticker Provider XNYS APLE <null> Equity XNYS XNYS Apple Hospitality Reit Inc USD US [ { { "tick": 0.0001 }: Price: 0, Tick: 0.0001 }, { { "price": 1, "tick": 0.01 }: Price: 1, Tick: 0.01 } ] [ Russell 2000, Russell 3000 ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1 16.11 <null> Year Month Day 2022 10 24 25 [ { [Idc|558, 558|APLE]: Key: Idc|558, Value: 558|APLE }, { [Figi, BBG006473QX9]: Key: Figi, Value: BBG006473QX9 }, { [Figic, BBG006473QX9]: Key: Figic, Value: BBG006473QX9 }, { [Bloomberg, APLE US Equity]: Key: Bloomberg, Value: APLE US Equity }, { [FIGI, BBG008P33727]: Key: FIGI, Value: BBG008P33727 }, { [FIGIC, BBG006473QX9]: Key: FIGIC, Value: BBG006473QX9 }, { [Valoren, 28266921]: Key: Valoren, Value: 28266921 }, { [CIK, 0001418121]: Key: CIK, Value: 0001418121 }, { [MorningStar, 0P000162G2]: Key: MorningStar, Value: 0P000162G2 }, { [Yahoo, APLE]: Key: Yahoo, Value: APLE } ] APLE US Equity 3 Exchange Ticker Provider BATS APLE <null> Equity XNYS XCBO Apple Hospitality Reit Inc USD US [ { { "tick": 0.0001 }: Price: 0, Tick: 0.0001 }, { { "price": 1, "tick": 0.01 }: Price: 1, Tick: 0.01 } ] [ Russell 2000, Russell 3000 ] Seconds Nanos 34200 0 Seconds Nanos 57600 0 ET 1 1 15.01 44277 Year Month Day 2022 7 11 1 [ { [Idc|729, 729|APLE]: Key: Idc|729, Value: 729|APLE }, { [Figi, BBG008P337J9]: Key: Figi, Value: BBG008P337J9 }, { [Figic, BBG006473QX9]: Key: Figic, Value: BBG006473QX9 }, { [Bloomberg, APLE UF Equity]: Key: Bloomberg, Value: APLE UF Equity } ] APLE UF Equity

##### 4.2.2 Equity - by MIC code

In [6]:
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Equity,
    Exchange = "XNGS", 
    Count = 3, // Select the first elements
    Start = 2 // Starting from element index
  )

let reply = service.StaticData(request, headers)
display(reply.Equities)

index Identifier Type Primary Operating Name Currency Country TickSizeRule Index Open Close TimeZone LotSize PointValue Price Volume Time Sources Mapping Bloomberg .. 0 Exchange Ticker Provider XNGS RELL <null> Equity XNGS XNAS Richardson Electronics Ltd USD US [ { { "tick": 0.0001 }: Price: 0, Tick: 0.0001 }, { { "price": 1, "tick": 0.01 }: Price: 1, Tick: 0.01 } ] Seconds Nanos 52200 0 Seconds Nanos 75600 0 1 1 <null> <null> Year Month Day 2022 10 24 29 [ { [Reuters, RELL.OQ]: Key: Reuters, Value: RELL.OQ }, { [CIK, 0000355948]: Key: CIK, Value: 0000355948 }, { [MorningStar, 0P000004OK]: Key: MorningStar, Value: 0P000004OK }, { [Esignal, RELL]: Key: Esignal, Value: RELL }, { [Idc|564, 564|RELL]: Key: Idc|564, Value: 564|RELL }, { [Figi, BBG000BS1WR4]: Key: Figi, Value: BBG000BS1WR4 }, { [Figic, BBG000BS1WR4]: Key: Figic, Value: BBG000BS1WR4 }, { [Bloomberg, RELL US Equity]: Key: Bloomberg, Value: RELL US Equity }, { [FIGI, BBG000BS24X8]: Key: FIGI, Value: BBG000BS24X8 }, { [FIGIC, BBG000BS1WR4]: Key: FIGIC, Value: BBG000BS1WR4 }, { [Valoren, 966560]: Key: Valoren, Value: 966560 } ] RELL US Equity 1 Exchange Ticker Provider XNGS IDXX <null> Equity XNGS XNAS Idexx Laboratories Inc USD US [ { { "tick": 0.0001 }: Price: 0, Tick: 0.0001 }, { { "price": 1, "tick": 0.01 }: Price: 1, Tick: 0.01 } ] [ Nasdaq 100, Nasdaq Composite, Russell 1000, Russell 3000, Snp 500 ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1 <null> <null> Year Month Day 2022 10 24 39 [ { [Reuters, IDXX.OQ]: Key: Reuters, Value: IDXX.OQ }, { [MorningStar, 0P000002TA]: Key: MorningStar, Value: 0P000002TA }, { [Yahoo, IDXX]: Key: Yahoo, Value: IDXX }, { [FIGI, BBG000BLS2X9]: Key: FIGI, Value: BBG000BLS2X9 }, { [FIGIC, BBG000BLRT07]: Key: FIGIC, Value: BBG000BLRT07 }, { [Valoren, 940695]: Key: Valoren, Value: 940695 }, { [CIK, 0000874716]: Key: CIK, Value: 0000874716 }, { [Esignal, IDXX]: Key: Esignal, Value: IDXX }, { [Idc|564, 564|IDXX]: Key: Idc|564, Value: 564|IDXX }, { [Figi, BBG000BLRT07]: Key: Figi, Value: BBG000BLRT07 }, { [Figic, BBG000BLRT07]: Key: Figic, Value: BBG000BLRT07 }, { [Bloomberg, IDXX US Equity]: Key: Bloomberg, Value: IDXX US Equity } ] IDXX US Equity 2 Exchange Ticker Provider XNGS SSRM <null> Equity XNGS XNAS Ssr Mining USD [ { { "tick": 0.0001 }: Price: 0, Tick: 0.0001 }, { { "price": 1, "tick": 0.01 }: Price: 1, Tick: 0.01 } ] [ Nasdaq Composite, Russell 1000, Russell 3000 ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1 <null> <null> Year Month Day 2022 10 24 31 [ { [FIGI, BBG000C0S772]: Key: FIGI, Value: BBG000C0S772 }, { [FIGIC, BBG000C0RWX9]: Key: FIGIC, Value: BBG000C0RWX9 }, { [Valoren, 37752727]: Key: Valoren, Value: 37752727 }, { [Esignal, SSRM]: Key: Esignal, Value: SSRM }, { [Reuters, SSRM.OQ]: Key: Reuters, Value: SSRM.OQ }, { [Idc|564, 564|SSRM]: Key: Idc|564, Value: 564|SSRM }, { [Figi, BBG000C0RWX9]: Key: Figi, Value: BBG000C0RWX9 }, { [Figic, BBG000C0RWX9]: Key: Figic, Value: BBG000C0RWX9 }, { [Bloomberg, SSRM US Equity]: Key: Bloomberg, Value: SSRM US Equity }, { [Yahoo, SSRM]: Key: Yahoo, Value: SSRM } ] SSRM US Equity

##### 4.2.3 Equity - by Ticker

In [7]:
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Equity,
    Ticker = "AAPL"
  )

let reply = service.StaticData(request, headers)
display(reply.Equities)

index Identifier Type Primary Operating Name Currency Country TickSizeRule Index Open Close TimeZone LotSize PointValue Price Volume Time Sources Mapping Bloomberg .. 0 Exchange Ticker Provider XNGS AAPL <null> Equity XNGS XNAS Apple Inc USD US [ { { "tick": 0.0001 }: Price: 0, Tick: 0.0001 }, { { "price": 1, "tick": 0.01 }: Price: 1, Tick: 0.01 } ] [ Dow Jones, Nasdaq 100, Nasdaq Composite, Russell 1000, Russell 3000, Snp 500 ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1 <null> <null> Year Month Day 2022 10 24 41 [ { [Yahoo, AAPL]: Key: Yahoo, Value: AAPL }, { [CIK, 0000320193]: Key: CIK, Value: 0000320193 }, { [FIGI, BBG000B9Y5X2]: Key: FIGI, Value: BBG000B9Y5X2 }, { [FIGIC, BBG000B9XRY4]: Key: FIGIC, Value: BBG000B9XRY4 }, { [Valoren, 908440]: Key: Valoren, Value: 908440 }, { [MorningStar, 0P000000GY]: Key: MorningStar, Value: 0P000000GY }, { [Esignal, AAPL]: Key: Esignal, Value: AAPL }, { [Idc|564, 564|AAPL]: Key: Idc|564, Value: 564|AAPL }, { [Figi, BBG000B9XRY4]: Key: Figi, Value: BBG000B9XRY4 }, { [Figic, BBG000B9XRY4]: Key: Figic, Value: BBG000B9XRY4 }, { [Bloomberg, AAPL US Equity]: Key: Bloomberg, Value: AAPL US Equity }, { [Reuters, AAPL.O]: Key: Reuters, Value: AAPL.O } ] AAPL US Equity 1 Exchange Ticker Provider BATS AAPL <null> Equity XNGS XCBO Apple Inc USD US [ { { "tick": 0.0001 }: Price: 0, Tick: 0.0001 }, { { "price": 1, "tick": 0.01 }: Price: 1, Tick: 0.01 } ] [ Dow Jones, Nasdaq 100, Nasdaq Composite, Russell 1000, Russell 3000, Snp 500 ] Seconds Nanos 34200 0 Seconds Nanos 57600 0 ET 1 1 144.96 3159611 Year Month Day 2022 7 11 1 [ { [Idc|729, 729|AAPL]: Key: Idc|729, Value: 729|AAPL }, { [Figi, BBG000B9Y6P9]: Key: Figi, Value: BBG000B9Y6P9 }, { [Figic, BBG000B9XRY4]: Key: Figic, Value: BBG000B9XRY4 }, { [Bloomberg, AAPL UF Equity]: Key: Bloomberg, Value: AAPL UF Equity } ] AAPL UF Equity

##### 4.2.4 Equity - by Codes

In [8]:
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Equity,
    Code = "US0378331005" // ISIN code
  )
let reply = service.StaticData(request, headers)
display(reply.Equities)

index Identifier Type Primary Operating Name Currency Country TickSizeRule Index Open Close TimeZone LotSize PointValue Price Volume Time Sources Mapping Bloomberg .. 0 Exchange Ticker Provider XNGS AAPL <null> Equity XNGS XNAS Apple Inc USD US [ { { "tick": 0.0001 }: Price: 0, Tick: 0.0001 }, { { "price": 1, "tick": 0.01 }: Price: 1, Tick: 0.01 } ] [ Dow Jones, Nasdaq 100, Nasdaq Composite, Russell 1000, Russell 3000, Snp 500 ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1 <null> <null> Year Month Day 2022 10 24 41 [ { [Yahoo, AAPL]: Key: Yahoo, Value: AAPL }, { [CIK, 0000320193]: Key: CIK, Value: 0000320193 }, { [FIGI, BBG000B9Y5X2]: Key: FIGI, Value: BBG000B9Y5X2 }, { [FIGIC, BBG000B9XRY4]: Key: FIGIC, Value: BBG000B9XRY4 }, { [Valoren, 908440]: Key: Valoren, Value: 908440 }, { [MorningStar, 0P000000GY]: Key: MorningStar, Value: 0P000000GY }, { [Esignal, AAPL]: Key: Esignal, Value: AAPL }, { [Idc|564, 564|AAPL]: Key: Idc|564, Value: 564|AAPL }, { [Figi, BBG000B9XRY4]: Key: Figi, Value: BBG000B9XRY4 }, { [Figic, BBG000B9XRY4]: Key: Figic, Value: BBG000B9XRY4 }, { [Bloomberg, AAPL US Equity]: Key: Bloomberg, Value: AAPL US Equity }, { [Reuters, AAPL.O]: Key: Reuters, Value: AAPL.O } ] AAPL US Equity 1 Exchange Ticker Provider BATS AAPL <null> Equity XNGS XCBO Apple Inc USD US [ { { "tick": 0.0001 }: Price: 0, Tick: 0.0001 }, { { "price": 1, "tick": 0.01 }: Price: 1, Tick: 0.01 } ] [ Dow Jones, Nasdaq 100, Nasdaq Composite, Russell 1000, Russell 3000, Snp 500 ] Seconds Nanos 34200 0 Seconds Nanos 57600 0 ET 1 1 144.96 3159611 Year Month Day 2022 7 11 1 [ { [Idc|729, 729|AAPL]: Key: Idc|729, Value: 729|AAPL }, { [Figi, BBG000B9Y6P9]: Key: Figi, Value: BBG000B9Y6P9 }, { [Figic, BBG000B9XRY4]: Key: Figic, Value: BBG000B9XRY4 }, { [Bloomberg, AAPL UF Equity]: Key: Bloomberg, Value: AAPL UF Equity } ] AAPL UF Equity

In [9]:
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Equity,
    Code = "BBG000B9Y6P9" // Bloomberg Figi code
  )

let reply = service.StaticData(request, headers)
display(reply.Equities)

index Identifier Type Primary Operating Name Currency Country TickSizeRule Index Open Close TimeZone LotSize PointValue Price Volume Time Sources Mapping Bloomberg .. 0 Exchange Ticker Provider BATS AAPL <null> Equity XNGS XCBO Apple Inc USD US [ { { "tick": 0.0001 }: Price: 0, Tick: 0.0001 }, { { "price": 1, "tick": 0.01 }: Price: 1, Tick: 0.01 } ] [ Dow Jones, Nasdaq 100, Nasdaq Composite, Russell 1000, Russell 3000, Snp 500 ] Seconds Nanos 34200 0 Seconds Nanos 57600 0 ET 1 1 144.96 3159611 Year Month Day 2022 7 11 1 [ { [Idc|729, 729|AAPL]: Key: Idc|729, Value: 729|AAPL }, { [Figi, BBG000B9Y6P9]: Key: Figi, Value: BBG000B9Y6P9 }, { [Figic, BBG000B9XRY4]: Key: Figic, Value: BBG000B9XRY4 }, { [Bloomberg, AAPL UF Equity]: Key: Bloomberg, Value: AAPL UF Equity } ] AAPL UF Equity

##### 4.2.5 Equity - by Ticker + MIC code

In [10]:
// The service also enables to query with multiple filters as follows
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Equity,
    Ticker = "AAPL",
    Exchange = "BATS"
  )

let reply = service.StaticData(request, headers)
display(reply.Equities)

index Identifier Type Primary Operating Name Currency Country TickSizeRule Index Open Close TimeZone LotSize PointValue Price Volume Time Sources Mapping Bloomberg .. 0 Exchange Ticker Provider BATS AAPL <null> Equity XNGS XCBO Apple Inc USD US [ { { "tick": 0.0001 }: Price: 0, Tick: 0.0001 }, { { "price": 1, "tick": 0.01 }: Price: 1, Tick: 0.01 } ] [ Dow Jones, Nasdaq 100, Nasdaq Composite, Russell 1000, Russell 3000, Snp 500 ] Seconds Nanos 34200 0 Seconds Nanos 57600 0 ET 1 1 144.96 3159611 Year Month Day 2022 7 11 1 [ { [Idc|729, 729|AAPL]: Key: Idc|729, Value: 729|AAPL }, { [Figi, BBG000B9Y6P9]: Key: Figi, Value: BBG000B9Y6P9 }, { [Figic, BBG000B9XRY4]: Key: Figic, Value: BBG000B9XRY4 }, { [Bloomberg, AAPL UF Equity]: Key: Bloomberg, Value: AAPL UF Equity } ] AAPL UF Equity

##### 4.2.6 Equity - index components

In [11]:
// Get an  index components
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Equity,
    Index = "NASDAQ 100",
    Count = 10
  )

let reply = service.StaticData(request, headers)
display(reply.Equities)

index Identifier Type Primary Operating Name Currency Country TickSizeRule Index Open Close TimeZone LotSize PointValue Price Volume Time Sources Mapping Bloomberg .. 0 Exchange Ticker Provider XLON 0XHS <null> Equity XLOM XLON Liberty Global Plc Liberty Global Ord Shs Class C USD GB [ { { "tick": 0.0005 }: Price: 0, Tick: 0.0005 }, { { "price": 1, "tick": 0.01 }: Price: 1, Tick: 0.01 }, { { "price": 2, "tick": 0.02 }: Price: 2, Tick: 0.02 }, { { "price": 5, "tick": 0.05 }: Price: 5, Tick: 0.05 }, { { "price": 10, "tick": 0.1 }: Price: 10, Tick: 0.1 }, { { "price": 20, "tick": 0.2 }: Price: 20, Tick: 0.2 }, { { "price": 50, "tick": 0.5 }: Price: 50, Tick: 0.5 }, { { "price": 100, "tick": 1 }: Price: 100, Tick: 1 }, { { "price": 200, "tick": 2 }: Price: 200, Tick: 2 }, { { "price": 500, "tick": 5 }: Price: 500, Tick: 5 }, { { "price": 1000, "tick": 10 }: Price: 1000, Tick: 10 }, { { "price": 2000, "tick": 20 }: Price: 2000, Tick: 20 }, { { "price": 5000, "tick": 50 }: Price: 5000, Tick: 50 }, { { "price": 10000, "tick": 100 }: Price: 10000, Tick: 100 }, { { "price": 20000, "tick": 200 }: Price: 20000, Tick: 200 }, { { "price": 50000, "tick": 500 }: Price: 50000, Tick: 500 } ] [ Nasdaq 100 ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1 22.885 <null> Year Month Day 2022 6 17 1 [ { [Idc|627, 627|E:0XHS]: Key: Idc|627, Value: 627|E:0XHS }, { [Figi, BBG00JRQT544]: Key: Figi, Value: BBG00JRQT544 }, { [Figic, BBG00JRQT535]: Key: Figic, Value: BBG00JRQT535 }, { [Bloomberg, 0XHS LN Equity]: Key: Bloomberg, Value: 0XHS LN Equity } ] 0XHS LN Equity 1 Exchange Ticker Provider BATS LILA <null> Equity XNGS XCBO Liberty Latin America Ltd USD BM [ { { "tick": 0.0001 }: Price: 0, Tick: 0.0001 }, { { "price": 1, "tick": 0.01 }: Price: 1, Tick: 0.01 } ] [ Nasdaq 100 ] Seconds Nanos 34200 0 Seconds Nanos 57600 0 ET 1 1 7 14423 Year Month Day 2022 7 11 1 [ { [Idc|729, 729|LILA]: Key: Idc|729, Value: 729|LILA }, { [Figi, BBG00JHP6H73]: Key: Figi, Value: BBG00JHP6H73 }, { [Figic, BBG00JHP6GP5]: Key: Figic, Value: BBG00JHP6GP5 }, { [Bloomberg, LILA UF Equity]: Key: Bloomberg, Value: LILA UF Equity } ] LILA UF Equity 2 Exchange Ticker Provider XLON 0LQQ <null> Equity XLOM XLON Vodafone Group Plc Vodafone Group Adr Representing 10 Ord S USD US [ { { "tick": 0.0005 }: Price: 0, Tick: 0.0005 }, { { "price": 1, "tick": 0.01 }: Price: 1, Tick: 0.01 }, { { "price": 2, "tick": 0.02 }: Price: 2, Tick: 0.02 }, { { "price": 5, "tick": 0.05 }: Price: 5, Tick: 0.05 }, { { "price": 10, "tick": 0.1 }: Price: 10, Tick: 0.1 }, { { "price": 20, "tick": 0.2 }: Price: 20, Tick: 0.2 }, { { "price": 50, "tick": 0.5 }: Price: 50, Tick: 0.5 }, { { "price": 100, "tick": 1 }: Price: 100, Tick: 1 }, { { "price": 200, "tick": 2 }: Price: 200, Tick: 2 }, { { "price": 500, "tick": 5 }: Price: 500, Tick: 5 }, { { "price": 1000, "tick": 10 }: Price: 1000, Tick: 10 }, { { "price": 2000, "tick": 20 }: Price: 2000, Tick: 20 }, { { "price": 5000, "tick": 50 }: Price: 5000, Tick: 50 }, { { "price": 10000, "tick": 100 }: Price: 10000, Tick: 100 }, { { "price": 20000, "tick": 200 }: Price: 20000, Tick: 200 }, { { "price": 50000, "tick": 500 }: Price: 50000, Tick: 500 } ] [ Nasdaq 100 ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1 15.6 200353 Year Month Day 2022 6 17 1 [ { [Idc|627, 627|E:0LQQ]: Key: Idc|627, Value: 627|E:0LQQ }, { [Figi, BBG00JRYC698]: Key: Figi, Value: BBG00JRYC698 }, { [Figic, BBG00JRYC689]: Key: Figic, Value: BBG00JRYC689 }, { [Bloomberg, 0LQQ LN Equity]: Key: Bloomberg, Value: 0LQQ LN Equity } ] 0LQQ LN Equity 3 Exchange Ticker Provider XLON 0Y9S <null> Equity XLOM XLON Check Point Software Technologies L Check Point Software Technologies Ord Sh USD IL [ { { "tick": 0.0001 }: Price: 0, Tick: 0.0001 }, { { "price": 1, "tick": 0.0002 }: Price: 1, Tick: 0.0002 }, { { "price": 2, "tick": 0.0005 }: Price: 2, Tick: 0.0005 }, { { "price": 5, "tick": 0.001 }: Price: 5, Tick: 0.001 }, { { "price": 10, "tick": 0.002 }: Price: 10, Tick: 0.002 }, { { "price": 20, "tick": 0.005 }: Price: 20,

##### 4.2.7 Equity - sectors

In [12]:
// Get equities by sectors
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Equity,
    EquitySector = "Electronic Computers",
    Count = 10
  )

let reply = service.StaticData(request, headers)
display(reply.Equities)

(empty)

#### 4.3 Retrieve reference data - Futures

To request for futures, the user must specify the AssetType as <i>`AssetType.Future`</i> in the static data request.

##### 4.3.1 Future - by Name

In [13]:
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Future,
    Name = "WTI Crude Futures"
  )

let reply = service.StaticData(request, headers)
reply.Futures
|> Seq.sortBy (fun f -> f.Maturity.Year * 16 + f.Maturity.Month )
|> display

index Identifier Type Primary Operating Name Currency Country TickSizeRule Index Open Close TimeZone LotSize PointValue Price Volume Time Sources Mapping Bloomberg .. 0 Exchange Ticker Provider IFEU WBSQ22 <null> Future IFEU IFEU Wti Crude Futures - Wti - Aug22 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 103.37 35461 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\Q22]: Key: Idc|756, Value: 756|F:WBS\Q22 }, { [Figi, BBG00DLK2233]: Key: Figi, Value: BBG00DLK2233 } ] 1 Exchange Ticker Provider IFEU WBSU22 <null> Future IFEU IFEU Wti Crude Futures - Wti - Sep22 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 100.48 38354 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\U22]: Key: Idc|756, Value: 756|F:WBS\U22 }, { [Figi, BBG00DLK2242]: Key: Figi, Value: BBG00DLK2242 } ] 2 Exchange Ticker Provider IFEU WBSV22 <null> Future IFEU IFEU Wti Crude Futures - Wti - Oct22 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 97.61 28725 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\V22]: Key: Idc|756, Value: 756|F:WBS\V22 }, { [Figi, BBG00DLK2251]: Key: Figi, Value: BBG00DLK2251 } ] 3 Exchange Ticker Provider IFEU WBSX22 <null> Future IFEU IFEU Wti Crude Futures - Wti - Nov22 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 95.05 14649 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\X22]: Key: Idc|756, Value: 756|F:WBS\X22 }, { [Figi, BBG00DLK2260]: Key: Figi, Value: BBG00DLK2260 } ] 4 Exchange Ticker Provider IFEU WBSZ22 <null> Future IFEU IFEU Wti Crude Futures - Wti - Dec22 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 92.99 20872 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\Z22]: Key: Idc|756, Value: 756|F:WBS\Z22 }, { [Figi, BBG005M8J3W4]: Key: Figi, Value: BBG005M8J3W4 } ] 5 Exchange Ticker Provider IFEU WBSF23 <null> Future IFEU IFEU Wti Crude Futures - Wti - Jan23 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 91.61 2648 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\F23]: Key: Idc|756, Value: 756|F:WBS\F23 }, { [Figi, BBG00DLK23S4]: Key: Figi, Value: BBG00DLK23S4 } ] 6 Exchange Ticker Provider IFEU WBSG23 <null> Future IFEU IFEU Wti Crude Futures - Wti - Feb23 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 90.07 1248 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\G23]: Key: Idc|756, Value: 756|F:WBS\G23 }, { [Figi, BBG00DLK23Y7]: Key: Figi, Value: BBG00DLK23Y7 } ] 7 Exchange Ticker Provider IFEU WBSH23 <null> Future IFEU IFEU Wti Crude Futures - Wti - Mar23 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 88.82 1415 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\H23]: Key: Idc|756, Value: 756|F:WBS\H23 }, { [Figi, BBG00FJJK500]: Key: Figi, Value: BBG00FJJK500 } ] 8 Exchange Ticker Provider IFEU WBSJ23 <null> Future IFEU IFEU Wti Crude Futures - Wti - Apr23 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 87.76 351 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\J23]: Key: Idc|756, Value: 756|F:WBS\J23 }, { [Figi, BBG00FJJK528]: Key: Figi, Value: BBG00FJJK528 } ] 9 Exchange Ticker Provider IFEU WBSK23 <null> Future IFEU IFEU Wti Crude Futures - Wti - May23 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 80.58 268 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\K23]: Key: Idc|756, Value: 756|F:WBS\K23 }, { [Figi, BBG00FJJK537]: Key: Figi, Value: BBG00FJJK537 } ] 10 Exchange Ticker Provider IFEU WBSM23 <null> Future IFEU IFEU Wti Crude Futures - Wti - Jun23 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 85.61 2649 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\M23

##### 4.3.2 Future - by Future contract

In [14]:
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Future,
    FutureContract = "WBS"
  )

let reply = service.StaticData(request, headers)
reply.Futures
|> Seq.sortBy (fun f -> f.Maturity.Year * 16 + f.Maturity.Month)
|> display

index Identifier Type Primary Operating Name Currency Country TickSizeRule Index Open Close TimeZone LotSize PointValue Price Volume Time Sources Mapping Bloomberg .. 0 Exchange Ticker Provider IFEU WBSQ22 <null> Future IFEU IFEU Wti Crude Futures - Wti - Aug22 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 103.37 35461 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\Q22]: Key: Idc|756, Value: 756|F:WBS\Q22 }, { [Figi, BBG00DLK2233]: Key: Figi, Value: BBG00DLK2233 } ] 1 Exchange Ticker Provider IFEU WBSU22 <null> Future IFEU IFEU Wti Crude Futures - Wti - Sep22 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 100.48 38354 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\U22]: Key: Idc|756, Value: 756|F:WBS\U22 }, { [Figi, BBG00DLK2242]: Key: Figi, Value: BBG00DLK2242 } ] 2 Exchange Ticker Provider IFEU WBSV22 <null> Future IFEU IFEU Wti Crude Futures - Wti - Oct22 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 97.61 28725 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\V22]: Key: Idc|756, Value: 756|F:WBS\V22 }, { [Figi, BBG00DLK2251]: Key: Figi, Value: BBG00DLK2251 } ] 3 Exchange Ticker Provider IFEU WBSX22 <null> Future IFEU IFEU Wti Crude Futures - Wti - Nov22 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 95.05 14649 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\X22]: Key: Idc|756, Value: 756|F:WBS\X22 }, { [Figi, BBG00DLK2260]: Key: Figi, Value: BBG00DLK2260 } ] 4 Exchange Ticker Provider IFEU WBSZ22 <null> Future IFEU IFEU Wti Crude Futures - Wti - Dec22 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 92.99 20872 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\Z22]: Key: Idc|756, Value: 756|F:WBS\Z22 }, { [Figi, BBG005M8J3W4]: Key: Figi, Value: BBG005M8J3W4 } ] 5 Exchange Ticker Provider IFEU WBSF23 <null> Future IFEU IFEU Wti Crude Futures - Wti - Jan23 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 91.61 2648 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\F23]: Key: Idc|756, Value: 756|F:WBS\F23 }, { [Figi, BBG00DLK23S4]: Key: Figi, Value: BBG00DLK23S4 } ] 6 Exchange Ticker Provider IFEU WBSG23 <null> Future IFEU IFEU Wti Crude Futures - Wti - Feb23 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 90.07 1248 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\G23]: Key: Idc|756, Value: 756|F:WBS\G23 }, { [Figi, BBG00DLK23Y7]: Key: Figi, Value: BBG00DLK23Y7 } ] 7 Exchange Ticker Provider IFEU WBSH23 <null> Future IFEU IFEU Wti Crude Futures - Wti - Mar23 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 88.82 1415 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\H23]: Key: Idc|756, Value: 756|F:WBS\H23 }, { [Figi, BBG00FJJK500]: Key: Figi, Value: BBG00FJJK500 } ] 8 Exchange Ticker Provider IFEU WBSJ23 <null> Future IFEU IFEU Wti Crude Futures - Wti - Apr23 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 87.76 351 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\J23]: Key: Idc|756, Value: 756|F:WBS\J23 }, { [Figi, BBG00FJJK528]: Key: Figi, Value: BBG00FJJK528 } ] 9 Exchange Ticker Provider IFEU WBSK23 <null> Future IFEU IFEU Wti Crude Futures - Wti - May23 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 80.58 268 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\K23]: Key: Idc|756, Value: 756|F:WBS\K23 }, { [Figi, BBG00FJJK537]: Key: Figi, Value: BBG00FJJK537 } ] 10 Exchange Ticker Provider IFEU WBSM23 <null> Future IFEU IFEU Wti Crude Futures - Wti - Jun23 USD GB [ { { "tick": 0.01 }: Price: 0, Tick: 0.01 } ] Seconds Nanos 0 0 Seconds Nanos 0 0 1 1000 85.61 2649 Year Month Day 2022 7 11 1 [ { [Idc|756, 756|F:WBS\M23

##### 4.3.3 Future - by Ticker

In [15]:
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Future,
    Ticker = "WBSH20"
  )

let reply = service.StaticData(request, headers)
reply.Futures
|> display

(empty)

##### 4.3.4 Future - by Category

In [16]:
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Future,
    FutureCategory = "Crude Oil",
    Count = 3
  )

let reply = service.StaticData(request, headers)
reply.Futures
|> display

(empty)